In [1]:
#| default_exp components/slider

In [2]:
#| export
from typing import Literal
import fasthtml.common as fh
from fasthtml.common import FT
from fastcore.all import *
from fastbite.core import *
from fastbite.components.base import *
from fastbite.components.base_styles import *
from fastbite.components.containers import *
from fastbite.components.media import *
from enum import Enum

In [3]:
#| export

def SliderContainer(*c, # Components
                    cls:Enum|str|tuple=(), # Additional classes on the container
                    type:Literal['slide', 'static']='slide', # Type of slider
                    id:str=None, # Optional ID for the carousel
                    **kwargs # Additional args for the container
                    ) -> FT:
    "Creates a slider container"
    id = id or 'default-carousel'
    base_cls = 'relative w-full'
    return fh.Div(
        *c,
        cls=(base_cls, stringify(cls)), 
        data_carousel=type,
        id=id,
        **kwargs
    )

class SliderItemT(VEnum):
    default = 'absolute block w-full -translate-x-1/2 -translate-y-1/2 top-1/2 left-1/2'
    linear = 'absolute block w-full -translate-x-1/2 -translate-y-1/2 top-1/2 left-1/2 duration-700 ease-linear'
    ease_out_in = 'absolute block w-full -translate-x-1/2 -translate-y-1/2 top-1/2 left-1/2 duration-700 ease-out-in'
    cubic_bezier = 'absolute block w-full -translate-x-1/2 -translate-y-1/2 top-1/2 left-1/2 duration-700 ease-out-in-cubic'

def SliderItem(*c, # Components to be displayed in the slide
                cls:Enum|str|tuple=SliderItemT.default, # Additional classes for the items' inner div
                outer_cls:Enum|str|tuple="hidden duration-700 ease-in-out", # Added outer_cls
                **kwargs # Additional args for the outer items div
                ) -> FT:
    "Creates a slider item"
    return fh.Div(
        fh.Div(*c, cls=stringify(cls)),  # Allow any content to be passed
        cls=stringify(outer_cls), # Use stringify for outer_cls
        data_carousel_item=True, 
        **kwargs
    )

def SliderItems(
        *c, # Components
        cls:Enum|str|tuple='', # Additional classes for the items container
        **kwargs # Additional args for the items container
    ) -> FT: # Div(..., cls='uk-slider-items uk-grid', ...)
    "Creates a slider items container"
    # This component seems unused/deprecated in favor of Slider structure?
    # Type hint added for consistency, but stringify isn't needed if unused.
    return Div(*c, cls=(stringify(cls)), **kwargs)

def SliderControls(
    cls:Enum|str|tuple='', # Additional classes for controls container (Not directly used on element)
    prev_button_cls:Enum|str|tuple='absolute top-0 start-0 z-30 flex items-center justify-center h-full px-4 cursor-pointer group focus:outline-none', # Added prev_button_cls
    next_button_cls:Enum|str|tuple='absolute top-0 end-0 z-30 flex items-center justify-center h-full px-4 cursor-pointer group focus:outline-none', # Added next_button_cls
    span_cls:Enum|str|tuple='inline-flex items-center justify-center w-10 h-10 rounded-full bg-white/30 dark:bg-gray-800/30 group-hover:bg-white/50 dark:group-hover:bg-gray-800/60 group-focus:ring-4 group-focus:ring-white dark:group-focus:ring-gray-800/70 group-focus:outline-none', # Added span_cls
    icon_cls:Enum|str|tuple='text-gray-800 dark:text-gray-300', # Added icon_cls
    **kwargs # Additional args for controls (Not directly used)
) -> tuple:
    "Creates previous and next controls for the slider"
    prev_button = fh.Button(
        type='button',
        data_carousel_prev=True,
        cls=stringify(prev_button_cls) # Use stringify
    )(
        fh.Span(
            cls=stringify(span_cls) # Use stringify
        )(
            Icon('chevron-left', height=24, width=24, cls=stringify(icon_cls)), # Use stringify
            fh.Span('Previous', cls='sr-only')
        )
    )

    next_button = fh.Button(
        type='button',
        data_carousel_next=True,
        cls=stringify(next_button_cls) # Use stringify
    )(
        fh.Span(
            cls=stringify(span_cls) # Use stringify
        )(
            Icon('chevron-right', height=24, width=24, cls=stringify(icon_cls)), # Use stringify
            fh.Span('Next', cls='sr-only')
        )
    )
    
    return prev_button, next_button


def SliderNav(
        num_slides:int, # Number of slides
        cls:Enum|str|tuple='absolute z-30 flex -translate-x-1/2 bottom-5 left-1/2 space-x-3 rtl:space-x-reverse', # Additional classes for the navigation container
        button_cls:Enum|str|tuple='w-3 h-3 rounded-full', # Added button_cls
        **kwargs # Additional args for the navigation container
    ) -> FT:
    "Navigation indicators for Slider component"
    return Div(
        *[fh.Button(
            type='button',
            aria_current='true' if i == 0 else 'false',
            aria_label=f'Slide {i+1}',
            data_carousel_slide_to=f'{i}',
            cls=stringify(button_cls) # Use stringify
        ) for i in range(num_slides)],
        cls=(stringify(cls)),
        **kwargs
    )

def Slider(*items, # List of items to show in slider
          wrapper_cls:Enum|str|tuple='relative min-h-56 overflow-hidden rounded-lg md:min-h-96', # Classes for the wrapper div
          cls:Enum|str|tuple='', # Classes for slider container
          show_controls=True, # Whether to show navigation controls
          show_indicators=True, # Whether to show slide indicators
          static:bool=False, # Whether to show static slider
          id:str=None, # Optional ID for the carousel
          **kwargs # Additional args for slider container
    ) -> FT:
    "Creates a complete slider with items, controls, and indicators"
    
    # Create wrapper for slides
    wrapper = fh.Div(
        *[SliderItem(item) for item in items],
        Script("htmx.onLoad(function(content) {initCarousels();})"),
        cls=stringify(wrapper_cls), # Use stringify
    )
    
    components = [wrapper]
    
    # Add indicators if requested
    if show_indicators:
        components.append(SliderNav(num_slides=len(items)))
    
    # Add controls if requested
    if show_controls:
        prev_button, next_button = SliderControls()
        components.extend([prev_button, next_button])
    
    return SliderContainer(
        *components,
        type="static" if static else "slide",
        id=id,
        cls=cls,
        **kwargs
    )

In [4]:
#| hide
import nbdev; nbdev.nbdev_export()